In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os

In [2]:
data = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/data"

### Check total expression file

In [3]:
#expression_file = os.path.join(data, "raw_counts_gene_by_cell.tsv")
exp_file = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/tmp_dynamic/Subset24/expression0.tsv.gz"
#read expression mtx
expression_mtx = pd.read_csv(exp_file, sep="\t")
#check the mtx
expression_mtx.head()
#check the shape of the mtx
expression_mtx.shape

(13406, 501)

In [4]:
display(expression_mtx)

,Unnamed: 0,S4_AAACCCAAGCACGATG-1,S4_AAAGAACAGTAAACTG-1,S4_AAAGAACCATGATAGA-1,S4_AAAGGATCACGAAGAC-1,S4_AAAGGGCCATACTGAC-1,S4_AAAGTCCAGGATTACA-1,S4_AACAACCAGCTGCCAC-1,S4_AACAAGATCTCTTAAC-1,S4_AACAGGGCATCCGAGC-1,...,S4_TTGTTGTGTCATCAGT-1,S4_TTTACCATCGACACTA-1,S4_TTTAGTCAGGAGAATG-1,S4_TTTAGTCTCCACTAGA-1,S4_TTTATGCGTGAACGGT-1,S4_TTTCACATCTTCTGGC-1,S4_TTTCAGTAGACGCAGT-1,S4_TTTCAGTCAAGGTTGG-1,S4_TTTCGATAGTCATGCT-1,S4_TTTGATCGTACTAACC-1
0,Mrpl15,3.0,9.0,0.0,4.0,2.0,0.0,7.0,1.0,0.0,...,0.0,4.0,0.0,0.0,2.0,1.0,4.0,1.0,4.0,5.0
1,Lypla1,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2,Tcea1,1.0,3.0,2.0,7.0,2.0,2.0,5.0,1.0,2.0,...,1.0,1.0,3.0,1.0,1.0,1.0,3.0,4.0,0.0,2.0
3,Atp6v1h,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0
4,Rb1cc1,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13401,Gm4577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13402,Gm29417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13403,Pcdhga9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13404,Fbxo15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
expressed_genes = set(expression_mtx['Unnamed: 0'])
display(len(expressed_genes))

13406

### Prep base GRN

In [19]:
base_grn_file = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/Data/grn_files/base_GRN.csv"
base_grn = pd.read_csv(base_grn_file)

In [27]:
collapsed_grn = base_grn.groupby('gene_short_name').max().reset_index()
#drop columns that are not needed
collapsed_grn = collapsed_grn.drop(columns=['Unnamed: 0', 'peak_id'])
# convert to boolean df (True/False) instead of 1/0
binlinking_mouse_base = (collapsed_grn.set_index('gene_short_name') > 0).reset_index()
#transpose the df
binlinking_mouse_base = binlinking_mouse_base.T
# Reset index to convert it to a column
binlinking_mouse_base = binlinking_mouse_base.reset_index()
# Set the first row as column headers
binlinking_mouse_base.columns = binlinking_mouse_base.iloc[0]
# Remove the first row
binlinking_mouse_base = binlinking_mouse_base[1:].reset_index(drop=True)
# Rename just the first column to empty string, keep others as they are
binlinking_mouse_base.columns = [''] + list(binlinking_mouse_base.columns[1:])


#### All TFs must be in the targets

In [28]:
# Check overlap between TFs (rows) and targets (columns)
tf_names = set(binlinking_mouse_base.iloc[:, 0])  # First column = TF names
target_names = set(binlinking_mouse_base.columns[1:])  # All other columns = targets

print(f"Total TFs: {len(tf_names)}")
print(f"Total targets: {len(target_names)}")

# Find TFs that are NOT in targets
tfs_not_in_targets = tf_names - target_names
print(f"TFs NOT in target columns: {len(tfs_not_in_targets)}")
if len(tfs_not_in_targets) > 0:
    print(f"Examples: {list(tfs_not_in_targets)[:10]}")

Total TFs: 1093
Total targets: 21256
TFs NOT in target columns: 354
Examples: ['Znf281', 'Ensg00000236207', 'Dmrt6', 'Klf10', 'Znf75a', 'Fra2', 'Pax4', 'Znf449', 'P53', 'Znf148']


In [29]:
# Filter: keep only TFs that ARE in the target columns
binlinking_mouse_base = binlinking_mouse_base[binlinking_mouse_base.iloc[:, 0].isin(target_names)]
# Reset index to be sequential
binlinking_mouse_base = binlinking_mouse_base.reset_index(drop=True)
print(f"After filtering shape: {binlinking_mouse_base.shape}")

After filtering shape: (739, 21257)


In [30]:
display(binlinking_mouse_base)

,,0610005C13Rik,0610007C21Rik,0610007L01Rik,0610007P08Rik,0610007P22Rik,0610008F07Rik,0610009B14Rik,0610009B22Rik,0610009D07Rik,...,snoR38,snoU105B,snoU109,snoU13,snoU54,snoU6-77,snoU85,snoU89,snoZ39,snosnR60_Z15
0,Ahr,True,True,False,False,False,False,True,True,True,...,True,True,False,False,True,True,False,True,False,False
1,Ahrr,True,True,False,False,False,False,True,True,True,...,True,False,False,False,True,True,False,True,False,False
2,Aire,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,Alx1,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,Alx3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734,Zkscan1,False,True,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
735,Zkscan3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
736,Zkscan4,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
737,Zscan10,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


#### Set self regulatory edges to False

In [31]:
#print sum of all values in binlinking_mouse_base
print(f'Sum of all values in binlinking_mouse_base: {binlinking_mouse_base.iloc[:, 1:].sum().sum()}')
# Set first column as index temporarily to work with gene names
tf_names = binlinking_mouse_base.iloc[:, 0].values
for i, tf in enumerate(tf_names):
    if tf in binlinking_mouse_base.columns[1:]:  # If TF appears as a target
        # Set the self-regulation to False
        binlinking_mouse_base.loc[i, tf] = False
# print sum of all values in binlinking_mouse_base after setting self-regulation to False
print(f'Sum of all True values after removing self-regulation: {binlinking_mouse_base.iloc[:, 1:].sum().sum()}')

Sum of all values in binlinking_mouse_base: 5246984
Sum of all True values after removing self-regulation: 5246655


In [32]:
#save the binlinking_mouse_base
binlinking_mouse_base.to_csv(os.path.join(data, "mouse_base_grn.tsv.gz"), sep="\t", index=False)


#### Check binlinking

In [35]:
binlinking_file = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/tmp_dynamic/Subset15/binlinking.tsv.gz"
# read binlinking file
binlinking = pd.read_csv(binlinking_file, sep="\t")

In [36]:
display(binlinking)

,Unnamed: 0,0610005C13Rik,0610009B22Rik,0610009O20Rik,0610010F05Rik,0610010K14Rik,0610012G03Rik,0610030E20Rik,0610037L13Rik,0610039K10Rik,...,Zufsp,Zw10,Zwilch,Zwint,Zxdc,Zyg11b,Zyx,Zzef1,Zzz3,a
0,Ahr,True,True,False,True,True,False,False,False,True,...,False,False,False,False,False,False,True,False,True,True
1,Aire,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
2,Ar,True,True,True,False,True,False,True,True,True,...,False,False,True,True,False,True,False,False,True,True
3,Arhgef12,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Arid2,False,True,True,False,True,True,True,False,True,...,True,False,True,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,Zfp82,False,False,True,True,True,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
402,Zfx,True,True,False,True,True,False,False,False,True,...,True,False,True,False,True,True,True,True,True,True
403,Zkscan1,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
404,Zkscan3,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False


In [37]:
# Check overlap between TFs (rows) and targets (columns)
tf_names = set(binlinking.iloc[:, 0])  # First column = TF names
target_names = set(binlinking.columns[1:])  # All other columns = targets

print(f"Total TFs: {len(tf_names)}")
print(f"Total targets: {len(target_names)}")

# Find TFs that are NOT in targets
tfs_not_in_targets = tf_names - target_names
print(f"TFs NOT in target columns: {len(tfs_not_in_targets)}")
if len(tfs_not_in_targets) > 0:
    print(f"Examples: {list(tfs_not_in_targets)[:10]}")

Total TFs: 406
Total targets: 10816
TFs NOT in target columns: 0


### Debug network reconstruct

In [38]:
import os
import sys
from subprocess import Popen
from dictys.network import reconstruct

In [39]:
input_expression = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/tmp_dynamic/Subset1/expression0.tsv.gz"
input_binlinking = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/tmp_dynamic/Subset1/binlinking.tsv.gz"
output_net_weight = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/tmp_dynamic/Subset1/net_weight.tsv.gz"
output_net_meanvar = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/tmp_dynamic/Subset1/net_meanvar.tsv.gz"
output_net_covfactor = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/tmp_dynamic/Subset1/net_covfactor.tsv.gz"
output_net_loss = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/tmp_dynamic/Subset1/net_loss.tsv.gz"
output_net_stats = "/ocean/projects/cis240075p/asachan/datasets/B_Cell/T_cell/outs/dictys/rbpj_ntc/tmp_dynamic/Subset1/net_stats.tsv.gz"

In [40]:
reconstruct(
            fi_exp=input_expression,
            fi_mask=input_binlinking,
            fo_weight=output_net_weight,
            fo_meanvar=output_net_meanvar,
            fo_covfactor=output_net_covfactor,
            fo_loss=output_net_loss,
            fo_stats=output_net_stats,
            device='cuda',  # Let CUDA use the GPU that SLURM assigned
            nth=5
        )

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [7]:
# Step 1: Load the data (exactly as reconstruct does)
print("=" * 60)
print("STEP 1: Loading data")
print("=" * 60)

dt0 = pd.read_csv(input_expression, header=0, index_col=0, sep='\t')
print(f"Expression matrix shape: {dt0.shape}")
print(f"Expression genes (first 10): {list(dt0.index[:10])}")
print()

mask = pd.read_csv(input_binlinking, header=0, index_col=0, sep='\t')
print(f"Binlinking mask shape: {mask.shape}")
print(f"Mask TFs/rows (first 10): {list(mask.index[:10])}")
print(f"Mask targets/columns (first 10): {list(mask.columns[:10])}")
print()

# Step 2: Check assertions
print("=" * 60)
print("STEP 2: Checking assertions")
print("=" * 60)

set_mask_index = set(mask.index)
set_dt0_index = set(dt0.index)
set_mask_columns = set(mask.columns)

# First assertion
diff1 = set_mask_index - set_dt0_index
print(f"TFs in mask but NOT in expression: {len(diff1)}")
if len(diff1) > 0:
    print(f"Examples: {list(diff1)[:10]}")
print(f"Assertion 1: len(set(mask.index)-set(dt0.index))==0 → {len(diff1)==0}")
print()

# Second assertion
diff2 = set_mask_columns - set_dt0_index
print(f"Target genes in mask but NOT in expression: {len(diff2)}")
if len(diff2) > 0:
    print(f"Examples: {list(diff2)[:10]}")
print(f"Assertion 2: len(set(mask.columns)-set(dt0.index))==0 → {len(diff2)==0}")
print()

# Step 3: Build unified gene list (where the error occurs)
print("=" * 60)
print("STEP 3: Building unified gene list")
print("=" * 60)

namereg = mask.index  # TF names
nametarget = mask.columns  # Target names
nreg, ntarget = len(namereg), len(nametarget)

print(f"namereg (TFs): {nreg}")
print(f"nametarget (all targets): {ntarget}")
print()

t1 = set(namereg)  # Set of all TFs
print(f"Set of TFs: {len(t1)}")
print()

# Targets that are NOT TFs
targets_not_tfs = list(filter(lambda x: x not in t1, nametarget))
print(f"Targets that are NOT TFs: {len(targets_not_tfs)}")
print()

# Build unified list: [all TFs] + [targets that aren't TFs]
namet = np.r_[namereg, targets_not_tfs]
print(f"namet (unified gene list): {len(namet)}")
print()

# THE FAILING ASSERTION
print("=" * 60)
print("THE ASSERTION THAT FAILS:")
print("=" * 60)
print(f"len(namet) = {len(namet)}")
print(f"ntarget = {ntarget}")
print(f"assert len(namet)==ntarget → {len(namet)==ntarget}")
print()

if len(namet) != ntarget:
    print("❌ ASSERTION FAILS!")
    print()
    print("DIAGNOSIS:")
    print("-" * 60)
    
    # Find TFs that appear as targets
    tfs_as_targets = set(namereg) & set(nametarget)
    print(f"TFs that also appear as targets: {len(tfs_as_targets)}")
    print(f"Examples: {list(tfs_as_targets)[:10]}")
    print()
    
    # Find targets that are TFs
    targets_that_are_tfs = set(nametarget) & set(namereg)
    print(f"Targets that are also TFs: {len(targets_that_are_tfs)}")
    print()
    
    # What should happen
    print(f"Expected: ntarget = nreg + (targets that aren't TFs)")
    print(f"Expected: {ntarget} = {nreg} + {len(targets_not_tfs)}")
    print(f"But namet = {len(namet)} (just TFs + non-TF targets)")
    print()
    print("The issue: Some TFs in columns are missing from rows!")
else:
    print("✅ ASSERTION PASSES!")


STEP 1: Loading data
Expression matrix shape: (13415, 500)
Expression genes (first 10): ['Mrpl15', 'Lypla1', 'Gm37988', 'Tcea1', 'Atp6v1h', 'Rb1cc1', '4732440D04Rik', 'Pcmtd1', 'Gm26901', 'Rrs1']

Binlinking mask shape: (405, 10793)
Mask TFs/rows (first 10): ['Ahr', 'Aire', 'Ar', 'Arhgef12', 'Arid2', 'Arid3a', 'Arid3b', 'Arid5a', 'Arid5b', 'Arnt']
Mask targets/columns (first 10): ['0610009B22Rik', '0610009O20Rik', '0610010F05Rik', '0610010K14Rik', '0610012G03Rik', '0610030E20Rik', '0610037L13Rik', '0610039K10Rik', '0610040B10Rik', '0610040F04Rik']

STEP 2: Checking assertions
TFs in mask but NOT in expression: 0
Assertion 1: len(set(mask.index)-set(dt0.index))==0 → True

Target genes in mask but NOT in expression: 0
Assertion 2: len(set(mask.columns)-set(dt0.index))==0 → True

STEP 3: Building unified gene list
namereg (TFs): 405
nametarget (all targets): 10793

Set of TFs: 405

Targets that are NOT TFs: 10388

namet (unified gene list): 10793

THE ASSERTION THAT FAILS:
len(namet) = 10